In [ ]:
def get_cur_hedons():
    """Return number of accumulated hedons."""
    return hedons


def get_cur_health():
    """Return the current health points."""
    return health


def offer_star(activity):
    """Simulates a offering the user a star for engaging in the exercise activity."""
    global star, last_star_offered, last_star_offered2

    last_star_offered, last_star_offered2, last_star_offered3 = time, last_star_offered, last_star_offered2
    star[1] = True
    star[0] = activity

def perform_activity(activity, duration):
    """Simulates the user’s performing activity activity for duration minutes."""

    global hedons, health, time, prev_activity, tired, star, last_tedious_work, prev_ran_duration


    if time - last_tedious_work < 120:
      tired = True

    time += duration

    has_right_star = star_can_be_taken(activity)
    star[1] = False
    star[0] = ""

    if activity == "resting":
      tired = False
      prev_ran_duration = 0
    elif activity == "running":
      if duration + prev_ran_duration > 180:
          if prev_ran_duration < 180:
            temp = 180-prev_ran_duration
            if temp <= duration:
              health += (duration - temp) + temp*3
            else:
              health += duration*3
          else:
            health += duration
      else:
        health += duration*3
      if has_right_star and not tired:
        if duration > 10:
          hedons += 50 - (duration-10)*2
        else:
          hedons += 5*duration
      elif has_right_star and tired:
        if duration > 10:
          hedons += 10 - 2*(duration-10)
        else:
          hedons += duration
      elif not has_right_star and not tired:
        if duration > 10:
          hedons += 20 - 2*(duration-10)
        else:
          hedons += 2*duration
      elif not has_right_star and tired:
        hedons -= 2*duration
      tired = True
      last_tedious_work = time
      prev_ran_duration += duration
    elif activity == "textbooks":
      health += duration*2
      if has_right_star and not tired:
        if duration > 20:
            hedons += 70 - duration
        elif duration > 10:
            hedons += 30 + duration
        else:
            hedons += 3*duration
      elif has_right_star and tired:
        hedons += duration
      elif not has_right_star and not tired:
        if duration > 20:
          hedons += 20 - (duration-20)
        else:
          hedons += duration
      elif not has_right_star and tired:
        hedons -= 2*duration
      tired = True
      last_tedious_work = time

    prev_activity = activity


def star_can_be_taken(activity):
    """Returns True iff a star can be used to get more hedons for activity activity."""
    return activity == star[0] and star[1] and time - last_star_offered3 > 120

def most_fun_activity_minute():
    """Returns the activity which would give the most hedons if the person
    performed it for one minute at the current time."""
    global tired, star

    running_nextmin = 0
    resting_nextmin = 0
    textbooks_nextmin = 0

    if tired:
      if star[1]:
        if star[0] == "running":
          running_nextmin = 1
        elif star[0] == "textbooks":
          textbooks_nextmin = 1
      else:
        running_nextmin = -2
        textbooks_nextmin = -2
    else:
      if star[1]:
        if star[0] == "running" and (prev_activity == "running" or prev_activity == "textbooks"):
          running_nextmin = 1
        elif star[0] == "textbooks" and prev_activity == "textbooks":
          textbooks_nextmin = 1
        elif star[0] == "running" and (prev_activity != "running" or prev_activity != "textbooks"):
          running_nextmin = 3+2
        elif star[0] == "textbooks" and (prev_activity != "running" or prev_activity != "textbooks"):
          textbooks_nextmin = 3+1
      else:
        running_nextmin = 2
        textbooks_nextmin = 1

    maximum = max(running_nextmin, resting_nextmin, textbooks_nextmin)
    if maximum == running_nextmin:
      return "running"
    elif maximum == resting_nextmin:
      return "resting"
    else:
      return "textbooks"

def initialize():
    global hedons, health, time, prev_activity, tired, star, last_star_offered, last_star_offered2, last_tedious_work, last_star_offered3, prev_ran_duration
    time = 0Gradescope

    print(most_fun_activity_minute()) # running
    perform_activity("textbooks", 30)
    print(get_cur_health()) # 150 = 90 + 30*2
    print(get_cur_hedons()) # -80 = -20 + 30 * (-2)
    offer_star("running")
    perform_activity("running", 20)
    print(get_cur_health()) # 210 = 150 + 20 * 3
    print(get_cur_hedons()) # -90 = -80 + 10 * (3-2) + 10 * (-2)
    perform_activity("running", 170)
    print(get_cur_health()) # 700 = 210 + 160 * 3 + 10 * 1
    print(get_cur_hedons()) # -430 = -90 + 170 * (-2)
    '''

True


In [ ]:
# new
def get_cur_hedons():
    """Return number of accumulated hedons."""
    return hedons


def get_cur_health():
    """Return the current health points."""
    return health


def offer_star(activity):
    """Simulates a offering the user a star for engaging in the exercise activity."""
    global star_log
    star_log.append([activity, True, time])

def perform_activity(activity, duration):
    """Simulates the user’s performing activity activity for duration minutes."""
    global hedons, health, time, activity_log, star_log, bored
    time += duration
    tired = is_tired()
    activity_log.append([activity, duration])


    # find number of stars in past 120 mins
    if not bored:
      elapsed = 0
      stars = 0
      for i in range (len(star_log) - 1, 0, -1):
        elapsed += time - star_log[i][2]
        if elapsed > 120:
          break
        stars += 1
      if stars >= 3:
        bored = True

    # Check if they have the right star
    if len(star_log) != 0:
      has_right_star = star_log[-1][0] == activity and star_log[-1][1]
      if has_right_star:
        star_log[-1][1] = False
    else:
      has_right_star = False

    # Checking previous ran duration
    if activity == "running":
      prev_time_ran = 0
      for i in range(len(activity_log)-2, 0, -1):
          if activity_log[i][0] == 'resting':
              break

          if activity_log[i][0] == 'running':
              prev_time_ran += activity_log[i][1]

    # Health
    if activity == "running":
      if duration + prev_time_ran > 180:
        if prev_time_ran < 180:
          temp = 180-prev_time_ran
          if temp <= duration:
            health += (duration - temp) + temp*3
          else:
            health += duration*3
        else:
          health += duration
      else:
        health += 3*duration
    elif activity == "textbooks":
      health += 2*duration

    # Hedons
    if bored or not has_right_star:
      if not tired:
        if activity == "running":
          if duration > 10:
            hedons += 20 - 2*(duration-10)
          else:
            hedons += 2*duration
        elif activity == "textbooks":
          if duration > 20:
            hedons += 20 - (duration-20)
          else:
            hedons += duration
      elif tired and (activity == "running" or activity == "textbooks"):
        hedons += (-2)*duration
    else:
      star_log[-1][1] = False
      if not tired:
        if activity == "running":
          if duration > 10:
            hedons += 50 - (duration-10)*2
          else:
            hedons += 5*duration
        elif activity == "textbooks":
          if duration > 20:
              hedons += 70 - duration
          elif duration > 10:
              hedons += 30 + duration
          else:
              hedons += 3*duration
      else:
        if activity == "running":
          if duration > 10:
            hedons += 10 - 2*(duration-10)
          else:
            hedons += duration
        elif activity == "textbooks":
          hedons += duration

def star_can_be_taken(activity):
    """Returns True iff a star can be used to get more hedons for activity activity."""
    global star_log
    if len(star_log) != 0:
      return star_log[-1][0] == activity and star_log[-1][1]
    else:
      return False

def most_fun_activity_minute():
    """Returns the activity which would give the most hedons if the person
    performed it for one minute at the current time."""
    global bored
    tired = is_tired()

    if bored and tired:
      return "resting"
    elif bored:
      return "running"
    else:
      if star_can_be_taken("running") and not tired:
        return "running"
      elif star_can_be_taken("textbooks") and not tired:
        return "textbooks"
      elif not tired:
        return "running"
      elif tired and star_can_be_taken("running"):
        return "running"
      elif tired and star_can_be_taken("textbooks"):
        return "textbooks"
      else:
        return "resting"

def is_tired():
    global activity_log
    time_counter = 120
    if len(activity_log) == 1:
      return activity_log[0][0] == "running" or activity_log[0][0] == "textbooks"
    for i in range(len(activity_log)-1, -1, -1):
      activity_i = activity_log[i][0]
      if time_counter > 0 and (activity_i == "running" or activity_i == "textbooks"):
        return True
      elif activity_i == "resting" and time_counter > 0:
        time_counter -= activity_log[i][1]
      elif time_counter <= 0:
        return False
    return not time_counter > 0



def initialize():
    global hedons, health, time, activity_log, star_log, bored
    bored = False
    time, hedons, health = 0, 0, 0
    activity_log = []
    tired = False
    star_log = []

if __name__ == '__main__':
    initialize()

    perform_activity("running", 30)
    print(get_cur_hedons()) # -20 = 10 * 2 + 20 * (-2)
    print(get_cur_health()) # 90 = 30 * 3
    print(most_fun_activity_minute()) #resting
    perform_activity("resting", 30)
    offer_star("running")
    print(most_fun_activity_minute()) # running
    perform_activity("textbooks", 30)
    print(get_cur_health()) # 150 = 90 + 30*2
    print(get_cur_hedons()) # -80 = -20 + 30 * (-2)
    offer_star("running")
    perform_activity("running", 20)
    print(get_cur_health()) # 210 = 150 + 20 * 3
    print(get_cur_hedons()) # -90 = -80 + 10 * (3-2) + 10 * (-2)
    perform_activity("running", 170)
    print(get_cur_health()) # 700 = 210 + 160 * 3 + 10 * 1
    print(get_cur_hedons()) # -430 = -90 + 170 * (-2)

In [ ]:
# new finished
def get_cur_hedons():
    """Return number of accumulated hedons."""
    return hedons


def get_cur_health():
    """Return the current health points."""
    return health


def offer_star(activity):
    """Simulates a offering the user a star for engaging in the exercise activity."""
    global star_log
    star_log.append([activity, True, time])

def perform_activity(activity, duration):
    """Simulates the user’s performing activity activity for duration minutes."""
    global hedons, health, time, activity_log, star_log, bored
    time += duration
    tired = is_tired()
    activity_log.append([activity, duration])


    # find number of stars in past 120 mins
    if not bored:
      elapsed = 0
      stars = 0
      for i in range (len(star_log) - 1, 0, -1):
        elapsed += time - star_log[i][2]
        if elapsed > 120:
          break
        stars += 1
      if stars >= 3:
        bored = True

    # Check if they have the right star
    if len(star_log) != 0:
      has_right_star = star_log[-1][0] == activity and star_log[-1][1]
      if has_right_star:
        star_log[-1][1] = False
    else:
      has_right_star = False

    # Checking previous ran duration
    if activity == "running":
      prev_time_ran = 0
      for i in range(len(activity_log)-2, 0, -1):
          if activity_log[i][0] == 'resting':
              break

          if activity_log[i][0] == 'running':
              prev_time_ran += activity_log[i][1]

    # Health
    if activity == "running":
      if duration + prev_time_ran > 180:
        if prev_time_ran < 180:
          temp = 180-prev_time_ran
          if temp <= duration:
            health += (duration - temp) + temp*3
          else:
            health += duration*3
        else:
          health += duration
      else:
        health += 3*duration
    elif activity == "textbooks":
      health += 2*duration

    # Hedons
    if bored or not has_right_star:
      if not tired:
        if activity == "running":
          if duration > 10:
            hedons += 20 - 2*(duration-10)
          else:
            hedons += 2*duration
        elif activity == "textbooks":
          if duration > 20:
            hedons += 20 - (duration-20)
          else:
            hedons += duration
      elif tired and (activity == "running" or activity == "textbooks"):
        hedons += (-2)*duration
    else:
      star_log[-1][1] = False
      if not tired:
        if activity == "running":
          if duration > 10:
            hedons += 50 - (duration-10)*2
          else:
            hedons += 5*duration
        elif activity == "textbooks":
          if duration > 20:
              hedons += 70 - duration
          elif duration > 10:
              hedons += 30 + duration
          else:
              hedons += 3*duration
      else:
        if activity == "running":
          if duration > 10:
            hedons += 10 - 2*(duration-10)
          else:
            hedons += duration
        elif activity == "textbooks":
          hedons += duration

def star_can_be_taken(activity):
    """Returns True iff a star can be used to get more hedons for activity activity."""
    global star_log, bored
    if len(star_log) != 0:
      return star_log[-1][0] == activity and star_log[-1][1] and not bored
    else:
      return False

def most_fun_activity_minute():
    """Returns the activity which would give the most hedons if the person
    performed it for one minute at the current time."""
    global bored
    tired = is_tired()

    if bored and tired:
      return "resting"
    elif bored:
      return "running"
    else:
      if star_can_be_taken("running") and not tired:
        return "running"
      elif star_can_be_taken("textbooks") and not tired:
        return "textbooks"
      elif not tired:
        return "running"
      elif tired and star_can_be_taken("running"):
        return "running"
      elif tired and star_can_be_taken("textbooks"):
        return "textbooks"
      else:
        return "resting"

def is_tired():
    global activity_log
    time_counter = 120
    if len(activity_log) == 1:
      return activity_log[0][0] == "running" or activity_log[0][0] == "textbooks"
    for i in range(len(activity_log)-1, -1, -1):
      activity_i = activity_log[i][0]
      if time_counter > 0 and (activity_i == "running" or activity_i == "textbooks"):
        return True
      elif activity_i == "resting" and time_counter > 0:
        time_counter -= activity_log[i][1]
      elif time_counter <= 0:
        return False
    return not time_counter > 0



def initialize():
    global hedons, health, time, activity_log, star_log, bored
    bored = False
    time, hedons, health = 0, 0, 0
    activity_log = []
    tired = False
    star_log = []

if __name__ == '__main__':
    initialize()

    perform_activity("running", 30)
    print(get_cur_hedons()) # -20 = 10 * 2 + 20 * (-2)
    print(get_cur_health()) # 90 = 30 * 3
    print(most_fun_activity_minute()) #resting
    perform_activity("resting", 30)
    offer_star("running")
    print(most_fun_activity_minute()) # running
    perform_activity("textbooks", 30)
    print(get_cur_health()) # 150 = 90 + 30*2
    print(get_cur_hedons()) # -80 = -20 + 30 * (-2)
    offer_star("running")
    perform_activity("running", 20)
    print(get_cur_health()) # 210 = 150 + 20 * 3
    print(get_cur_hedons()) # -90 = -80 + 10 * (3-2) + 10 * (-2)
    perform_activity("running", 170)
    print(get_cur_health()) # 700 = 210 + 160 * 3 + 10 * 1
    print(get_cur_hedons()) # -430 = -90 + 170 * (-2)

In [ ]:
#best
def get_cur_hedons():
    """Return number of accumulated hedons."""
    return hedons


def get_cur_health():
    """Return the current health points."""
    return health


def offer_star(activity):
    """Simulates a offering the user a star for engaging in the exercise activity."""
    global star, last_star_offered, last_star_offered2

    last_star_offered, last_star_offered2, last_star_offered3 = time, last_star_offered, last_star_offered2
    star[1] = True
    star[0] = activity

def perform_activity(activity, duration):
    """Simulates the user’s performing activity activity for duration minutes."""

    global hedons, health, time, prev_activity, tired, star, last_tedious_work, prev_ran_duration


    if time - last_tedious_work < 120:
      tired = True

    time += duration

    has_right_star = star_can_be_taken(activity)
    star[1] = False
    star[0] = ""

    if activity == "resting":
      tired = False
      prev_ran_duration = 0
    elif activity == "running":
      if duration + prev_ran_duration > 180:
          if prev_ran_duration < 180:
            temp = 180-prev_ran_duration
            if temp <= duration:
              health += (duration - temp) + temp*3
            else:
              health += duration*3
          else:
            health += duration
      else:
        health += duration*3
      if has_right_star and not tired:
        if duration > 10:
          hedons += 50 - (duration-10)*2
        else:
          hedons += 5*duration
      elif has_right_star and tired:
        if duration > 10:
          hedons += 10 - 2*(duration-10)
        else:
          hedons += duration
      elif not has_right_star and not tired:
        if duration > 10:
          hedons += 20 - 2*(duration-10)
        else:
          hedons += 2*duration
      elif not has_right_star and tired:
        hedons -= 2*duration
      tired = True
      last_tedious_work = time
      prev_ran_duration += duration
    elif activity == "textbooks":
      health += duration*2
      if has_right_star and not tired:
        hedons += 4*duration
      elif has_right_star and tired:
        hedons += duration
      elif not has_right_star and not tired:
        if duration > 20:
          hedons += 20 - (duration-20)
        else:
          hedons += duration
      elif not has_right_star and tired:
        hedons -= 2*duration
      tired = True
      last_tedious_work = time

    prev_activity = activity


def star_can_be_taken(activity):
    """Returns True iff a star can be used to get more hedons for activity activity."""
    return activity == star[0] and star[1] and time - last_star_offered3 > 120

def most_fun_activity_minute():
    """Returns the activity which would give the most hedons if the person
    performed it for one minute at the current time."""
    global tired, star

    running_nextmin = 0
    resting_nextmin = 0
    textbooks_nextmin = 0

    if tired:
      if star[1]:
        if star[0] == "running":
          running_nextmin = 1
        elif star[0] == "textbooks":
          textbooks_nextmin = 1
      else:
        running_nextmin = -2
        textbooks_nextmin = -2
    else:
      if star[1]:
        if star[0] == "running" and (prev_activity == "running" or prev_activity == "textbooks"):
          running_nextmin = 1
        elif star[0] == "textbooks" and prev_activity == "textbooks":
          textbooks_nextmin = 1
        elif star[0] == "running" and (prev_activity != "running" or prev_activity != "textbooks"):
          running_nextmin = 3+2
        elif star[0] == "textbooks" and (prev_activity != "running" or prev_activity != "textbooks"):
          textbooks_nextmin = 3+1
      else:
        running_nextmin = 2
        textbooks_nextmin = 1

    maximum = max(running_nextmin, resting_nextmin, textbooks_nextmin)
    if maximum == running_nextmin:
      return "running"
    elif maximum == resting_nextmin:
      return "resting"
    else:
      return "textbooks"

def initialize():
    global hedons, health, time, prev_activity, tired, star, last_star_offered, last_star_offered2, last_tedious_work, last_star_offered3, prev_ran_duration
    time = 0
    hedons = 0
    health = 0
    prev_activity = ""
    prev_ran_duration = 0
    tired = False
    star = ["", False]
    last_star_offered, last_star_offered2, last_tedious_work, last_star_offered3 = -10**1000, -10**1000, -10**1000, -10**1000

if __name__ == '__main__':
    initialize()
    offer_star("textbooks")
    start_hedons = 0
    perform_activity('textbooks', 1)
    print(get_cur_hedons() == start_hedons + 1 + 3)  # Base + star bonus
    '''
    perform_activity("running", 30)
    print(get_cur_hedons()) # -20 = 10 * 2 + 20 * (-2)
    print(get_cur_health()) # 90 = 30 * 3
    print(most_fun_activity_minute()) #resting
    perform_activity("resting", 30)
    offer_star("running")
    print(most_fun_activity_minute()) # running
    perform_activity("textbooks", 30)
    print(get_cur_health()) # 150 = 90 + 30*2
    print(get_cur_hedons()) # -80 = -20 + 30 * (-2)
    offer_star("running")
    perform_activity("running", 20)
    print(get_cur_health()) # 210 = 150 + 20 * 3
    print(get_cur_hedons()) # -90 = -80 + 10 * (3-2) + 10 * (-2)
    perform_activity("running", 170)
    print(get_cur_health()) # 700 = 210 + 160 * 3 + 10 * 1
    print(get_cur_hedons()) # -430 = -90 + 170 * (-2)
    '''

https://www.cs.toronto.edu/~guerzhoy/180/gamify_test_cases.html


In [ ]:
def estimate_health_delta(activity, duration):
    global running_time, last_activity

    health_gained = 0

    if activity == "running":
        if last_activity == "running":
            health_gained = (3 * (min(duration, max(0, 180 - running_time))) +
                             1 * (max(0, duration - min(duration, max(0, 180 - running_time))))) # The user gains 3 health points
            # for the first 180 minutes of running and 1 health point for the rest, but we have to consider if teh user runs for
            # less than 180 minutes
        else:
            health_gained = (3 * (min(duration, 180)) + 1 *
                             (max(0, duration - (min(duration, 180)))))
    elif activity == "textbooks":
        health_gained = 2 * duration
    else:
        health_gained = 0

    return health_gained


In [ ]:
def estimate_hedons_delta(activity, duration):
    '''Return the amount of hedons the user would get for performing activity
    activity for duration minutes'''

    global tiredness, last_tired_finished

    hedons_gained = 0

    tiredness = ((cur_time - last_tired_finished) < 120) # If the user has done something tiring within 2 hours, the
                                                         # will remain tired
    if activity == "running":
        if tiredness:
            hedons_gained = (-2) * duration
        else:
            hedons_gained = 2 * min(duration, 10) + (-2) * max(0, duration - 10) # The user gains 2 hedons per minute
            # for the first 10 minutes and then -2 hedons per minute for the rest

    elif activity == "textbooks":
        if tiredness:
            hedons_gained = -2 * duration
        else:
            hedons_gained = 1 * min(duration, 20) + (-1) * max(0, duration - 20) # The user gains 1 hedon per minute
            # for the first 20 minutes and then -1 hedons per minute for the rest

    elif activity == "resting":
        hedons_gained = 0 * duration

    return hedons_gained

In [ ]:
def initialize():
    global hedons, health, time, activity_log, star_log, bored
    hedons = 0
    health = 0
    time = 0
    activity_log = []   # (activity, duration, end_time, cont_run_time)
    star_log = []       # [(activity, offer_time)]
    bored = False


def get_cur_hedons():
    return hedons


def get_cur_health():
    return health


def is_tired():
    """The user is tired if they finished running or carrying textbooks less than 2h ago."""
    global activity_log, time
    for i in range(len(activity_log) - 1, -1, -1):
        act, dur, end_time, _ = activity_log[i]
        if act != "resting":
            if time - end_time < 120:
                return True
            else:
                return False
    return False


def offer_star(activity):
    """Offer a star for an activity at current time."""
    global star_log, bored, time
    star_log.append((activity, time))
    # Check if 3 stars offered in last 2h
    recent = [t for a, t in star_log if time - t <= 120]
    if len(recent) >= 3:
        bored = True


def star_can_be_taken(activity):
    global star_log, bored
    if bored:
        return False
    if not star_log:
        return False
    return star_log[-1][0] == activity


def perform_activity(activity, duration):
    global hedons, health, time, activity_log, bored, star_log

    tired = is_tired()
    cont_run_time = 0

    # Decide if we use a star
    use_star = False
    if star_can_be_taken(activity):
        use_star = True
        star_log.pop()  # star consumed immediately

    # Find previous continuous running time
    prev_run = 0
    if activity_log and activity_log[-1][0] == "running":
        prev_run = activity_log[-1][3]

    # ---- HEALTH ----
    if activity == "running":
        run_time = prev_run
        for m in range(duration):
            run_time += 1
            if run_time <= 180:
                health += 3
            else:
                health += 1
        cont_run_time = run_time
    elif activity == "textbooks":
        health += 2 * duration
        cont_run_time = 0
    else:  # resting
        cont_run_time = 0

    # ---- HEDONS ----
    for m in range(duration):
        if activity == "resting":
            hedons += 0
        elif tired:
            hedons += -2
        else:
            if activity == "running":
                hedons += 2 if m < 10 else -2
            elif activity == "textbooks":
                hedons += 1 if m < 20 else -1

        # Star bonus applies only for first 10 minutes
        if use_star and m < 10 and activity != "resting":
            hedons += 3

    time += duration
    activity_log.append((activity, duration, time, cont_run_time))


def most_fun_activity_minute():
    """Returns the activity that would give the most hedons if performed for 1 min now."""
    tired = is_tired()
    best_vals = {"resting": 0}

    # Running
    if tired:
        val = -2
    else:
        val = 2
    if star_can_be_taken("running"):
        val += 3
    best_vals["running"] = val

    # Textbooks
    if tired:
        val = -2
    else:
        val = 1
    if star_can_be_taken("textbooks"):
        val += 3
    best_vals["textbooks"] = val

    # Pick max
    best_act = max(best_vals, key=best_vals.get)
    return best_act

if __name__ == "__main__":
    initialize()

    perform_activity("running", 30)
    print(get_cur_hedons())  # -20
    print(get_cur_health())  # 90
    print(most_fun_activity_minute())  # resting

    perform_activity("resting", 30)
    offer_star("running")
    print(most_fun_activity_minute())  # running

    perform_activity("textbooks", 30)
    print(get_cur_health())  # 150
    print(get_cur_hedons())  # -80

    offer_star("running")
    perform_activity("running", 20)
    print(get_cur_health())  # 210
    print(get_cur_hedons())  # -90

    perform_activity("running", 170)
    print(get_cur_health())  # 700
    print(get_cur_hedons())  # -430



-20
90
resting
running
150
-80
210
-90
700
-400


In [ ]:
# Health & Hedons Simulation (simple version)

def initialize():
    global hedons, health, time, activity_log, star_log, bored
    hedons = 0
    health = 0
    time = 0
    activity_log = []   # each item: [activity, duration, end_time, cont_run_time]
    star_log = []       # each item: [activity, offer_time]
    bored = False


def get_cur_hedons():
    return hedons


def get_cur_health():
    return health


def is_tired():
    """User is tired if last non-resting activity finished < 120 mins ago."""
    global activity_log, time
    # look backwards in log
    for i in range(len(activity_log)-1, -1, -1):
        act = activity_log[i][0]
        end_time = activity_log[i][2]
        if act != "resting":
            if time - end_time < 120:
                return True
            else:
                return False
    return False


def offer_star(activity):
    global star_log, bored, time
    star_log.append([activity, time])
    # count stars in last 120 minutes
    count = 0
    for s in star_log:
        if time - s[1] <= 120:
            count += 1
    if count >= 3:
        bored = True


def star_can_be_taken(activity):
    global star_log, bored
    if bored:
        return False
    if len(star_log) == 0:
        return False
    # only the most recent star can be used
    return star_log[-1][0] == activity


def perform_activity(activity, duration):
    global hedons, health, time, activity_log, star_log

    tired = is_tired()

    # check star use
    use_star = False
    if star_can_be_taken(activity):
        use_star = True
        star_log.pop()  # remove star once used

    # continuous running time
    prev_run = 0
    if len(activity_log) > 0 and activity_log[-1][0] == "running":
        prev_run = activity_log[-1][3]

    cont_run_time = 0

    # --- Health ---
    if activity == "running":
        run_time = prev_run
        for m in range(duration):
            run_time += 1
            if run_time <= 180:
                health += 3
            else:
                health += 1
        cont_run_time = run_time
    elif activity == "textbooks":
        health += 2 * duration
        cont_run_time = 0
    else:  # resting
        cont_run_time = 0

    # --- Hedons ---
    for m in range(duration):
        if activity == "resting":
            hedons += 0
        elif tired:
            hedons += -2
        else:
            if activity == "running":
                if m < 10:
                    hedons += 2
                else:
                    hedons += -2
            elif activity == "textbooks":
                if m < 20:
                    hedons += 1
                else:
                    hedons += -1

        # star bonus for first 10 minutes only
        if use_star and m < 10 and activity != "resting":
            hedons += 3

    time += duration
    activity_log.append([activity, duration, time, cont_run_time])


def most_fun_activity_minute():
    tired = is_tired()

    # start with resting
    best_activity = "resting"
    best_value = 0

    # running
    if tired:
        val = -2
    else:
        val = 2
    if star_can_be_taken("running"):
        val += 3
    if val > best_value:
        best_value = val
        best_activity = "running"

    # textbooks
    if tired:
        val = -2
    else:
        val = 1
    if star_can_be_taken("textbooks"):
        val += 3
    if val > best_value:
        best_value = val
        best_activity = "textbooks"

    return best_activity


# -------------------------------
# Example from your test in __main__
# -------------------------------
if __name__ == "__main__":
    initialize()

    perform_activity("running", 30)
    print(get_cur_hedons()) # -20
    print(get_cur_health()) # 90
    print(most_fun_activity_minute()) # resting

    perform_activity("resting", 30)
    offer_star("running")
    print(most_fun_activity_minute()) # running

    perform_activity("textbooks", 30)
    print(get_cur_health()) # 150
    print(get_cur_hedons()) # -80

    offer_star("running")
    perform_activity("running", 20)
    print(get_cur_health()) # 210
    print(get_cur_hedons()) # -90

    perform_activity("running", 170)
    print(get_cur_health()) # 700
    print(get_cur_hedons()) # -430


-20
90
resting
running
150
-80
210
-90
700
-400
